# Récupération et nettoyage des données du modèle

In [1]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt 
import statistics

In [2]:
path1 = '/Users/geoffroyperonne/Desktop/Hashrate.csv'
path2 = '/Users/geoffroyperonne/Desktop/cours.csv'

In [3]:
df1 = pd.read_csv(path1,sep=';') 
df2 = pd.read_csv(path2,sep=';')

In [4]:
df1=df1.join(df1['DateTime'].str.split(' ',expand=True).rename(columns={0:'Date',1:'inutile',2:'Hashrate'}))
df1=df1.drop(['DateTime','inutile'],axis=1)
df2=df2.drop(['Ouv.','Plus Haut','Plus Bas','Vol.','Variation %'],axis=1)

In [5]:
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [6]:
df2['Dernier']=df2['Dernier'].replace('.','')

In [7]:
def nettoyer1(x):
    return(float(x.replace(',','.')))

def nettoyer2(x):
    return(float((x.replace('.','')).replace(',','.')))

def string(x):
    return(str(x))

def entier(x):
    return(int(x))

In [8]:
df1['Hash Rate']=df1['Hash Rate'].apply(nettoyer1)
df2['Dernier']=df2['Dernier'].apply(nettoyer2)

In [9]:
df=df1.merge(df2)
df=df.assign(BTC=0)

In [10]:
for i in range(575):
    df['BTC'][i]=3600

for i in range(575,2035):
    df['BTC'][i]=1800
    
for i in range(2035,2557):
    df['BTC'][i]=900

<ipython-input-10-5bf19dc8d0d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BTC'][i]=3600
<ipython-input-10-5bf19dc8d0d1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BTC'][i]=1800
<ipython-input-10-5bf19dc8d0d1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BTC'][i]=900


In [11]:
df=df.assign(Seuil=(df['BTC']*df['Dernier'])/(0.1*24)/(df['Hash Rate']*10**(-12)))

In [13]:
df['Seuil']=df['Seuil'].rolling(50).mean()

In [15]:
path = '/Users/geoffroyperonne/Desktop/equipement.csv'

In [16]:
df3 = pd.read_csv(path,sep=',')

In [17]:
df3=df3.drop(['Unnamed: 8','Unnamed: 9','Unnamed: 1','Unnamed: 3'],axis=1)

In [18]:
df3.drop(df3.index[0:13],0,inplace=True)

In [19]:
df3=df3.rename(columns = {'CBECI SHA-256 Mining Equipment List': 'Mineur', 'Unnamed: 2':'Date' 
                        , 'Unnamed: 1': 'Date de sortie','Unnamed: 4': 'Hashing power (Th/s)', 'Unnamed: 5': 'Puissance (W)', 'Unnamed: 6': 'Efficacité (J/Gh)', 'Unnamed: 7': 'Poids (kg)'})

In [20]:
def date(x):
    return('01.'+x)

In [21]:
df3['Date']=df3['Date'].apply(date)

In [22]:
def inverser(x):
    return(x[3:6]+x[0:3]+x[6:10])

In [23]:
df3['Date']=df3['Date'].apply(inverser)

In [24]:
df3['Date'] = pd.to_datetime(df3['Date'])

In [25]:
df=df.merge(df3,how='outer')

In [26]:
df['Efficacité (J/Gh)'] = df['Efficacité (J/Gh)'].fillna(0)

In [27]:
df['Efficacité (J/Gh)']=df['Efficacité (J/Gh)'].astype(float)

In [28]:
for i in range(2609):
    if df['Efficacité (J/Gh)'][i]==0.:
        df['Efficacité (J/Gh)'][i]=-10

<ipython-input-28-2fa8926b2a51>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Efficacité (J/Gh)'][i]=-10


In [29]:
resultat=[]
for i in range(2609):
    if df['Seuil'][i]>0:
        if df['Efficacité (J/Gh)'][i]>0:
            t=0
            while df['Efficacité (J/Gh)'][i]<df['Seuil'][t+i] and t+i<2608:
                t+=1
            resultat.append(t)
        
print(resultat)

[382, 906, 125, 692, 659, 528, 479, 505, 405, 446, 397, 384, 280, 216, 244, 185, 279, 209, 381, 251, 44, 123, 320, 299, 873, 288, 286, 225, 191, 150, 149, 225, 223, 89, 195, 192, 186, 127, 483, 453, 435, 156, 724, 718, 599, 418, 126, 354, 359, 0, 324, 327, 617, 443, 541, 540, 533, 532, 504, 378, 228, 439, 167, 102, 165, 396, 38, 371, 361, 221, 243, 67, 73, 308, 184, 283, 264, 277, 161, 150, 612, 137, 99, 40, 31, 12, 0, 0, 334, 245, 214, 213]


In [32]:
T = statistics.mean(resultat)
print("La durée de vie moyenne d'un appareil de minage est", T,"jours")

La durée de vie moyenne d'un appareil de minage est 305.8369565217391 jours
